In [21]:
%matplotlib inline
import sys
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arff

from sklearn import tree
from sklearn import linear_model
from sklearn.externals.six import StringIO  
from IPython.core.display import Image

import pydot

# Load the Data

In [22]:
%%time
gender = {'Male' : 0, 'Female' : 1, 'Unknown' : 2}
language = {'English' : 0, 'EnglishandAnother' : 1, 'Another' : 2}

trainfile = 'sample_data\\train05again.arff'

examples = []
features = []
labels = []

with open(trainfile, 'rb') as af:
    arffFile = arff.load(af)
    # do not include college gpa and college credits
    features = [arffFile['attributes'][0:-3][i][0].encode("ascii") for i in range(len(arffFile['attributes'][0:-3]))]
    data = arffFile['data']
    for row in data:
        row[0] = gender[row[0]] # index of gender
        row[12] = language[row[12]] # index of language
        examples.append(row[0:-3]) # do not include number of credits taken first year
        labels.append(row[-1])
        
print 'Number of examples:', len(examples)

Number of examples: 2883
Wall time: 120 ms


# Decision Tree

In [23]:
%%time
dt = tree.DecisionTreeClassifier()
dt = dt.fit(examples, labels)
print dt

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')
Wall time: 32 ms


In [24]:
%%time
dot_data = StringIO()
tree.export_graphviz(dt, out_file=dot_data, feature_names=features) 
#tree.export_graphviz(dt, out_file=dot_data) 
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
treepng = graph.create_png()

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.832749 to fit

Wall time: 10.7 s


In [25]:
Image(treepng)

In [26]:
%%time
with open('tree_SATACT.png', 'wb') as f:
    f.write(treepng)

Wall time: 0 ns


# Evaluate Tree on Test Set

In [27]:
%%time
trainfile = 'sample_data\\train05.arff'

testexamples = []
testlabels = []

with open(trainfile, 'rb') as af:
    arffFile = arff.load(af)
    # do not include college gpa and college credits
    features = [arffFile['attributes'][0:-3][i][0].encode("ascii") for i in range(len(arffFile['attributes'][0:-3]))]
    data = arffFile['data']
    for row in data:
        row[0] = gender[row[0]] # index of gender
        row[12] = language[row[12]] # index of language
        testexamples.append(row[0:-3]) # do not include number of credits taken first year
        testlabels.append(row[-1])

Wall time: 0 ns


In [28]:
%%time
numCorrect = 0
numIncorrect = 0
testCount = len(testexamples)
for index in range(testCount):
    x = testexamples[index]
    if dt.predict(x) == testlabels[index]:
        numCorrect += 1
    else:
        numIncorrect += 1
print 'Correctly Classified:', numCorrect
print 'Incorrectly Classified:', numIncorrect
print 'Error: %f' % (float(numCorrect) / float(testCount) * 100)

Correctly Classified: 82
Incorrectly Classified: 0
Error: 100.000000
Wall time: 4 ms
